In [1]:
%%time
import os
import threading
import subprocess
import numpy as np
from random import randrange
import math

CPU times: user 98.4 ms, sys: 14.1 ms, total: 113 ms
Wall time: 162 ms


In [2]:
source_dir = "../data/source/"
source_files = []
for j in sorted(os.listdir(source_dir)):
    source_files += [f"{source_dir}{j}"]
source_files

['../data/source/0.mkv',
 '../data/source/1.mkv',
 '../data/source/2.mkv',
 '../data/source/3.mkv']

In [3]:
# splitting into file_time (in seconds) +- delta_seconds
file_time = 5.0 # seconds
delta = 0.5 # seconds
delta_step = 0.01 # seconds
min_sr = 8000
max_sr = 48000
resulting_durations = np.around(np.arange(file_time - delta, file_time + delta, delta_step), decimals = 2)
sampling_rates = np.arange(min_sr, max_sr, 1)

# duration in seconds, nned to pass file.mkv as last argument
check_file_duration_command = "ffprobe -v error -show_entries format=duration -of default=noprint_wrappers=1:nokey=1"

# now generate list of ffmpeg commands:
ffmpeg_commands = []
for file_index, file in enumerate(source_files):
    file_duration = int(float(subprocess.getoutput(f"{check_file_duration_command} {file}")))
    start_times = np.around(np.arange(0, file_duration, math.ceil(file_time + delta)), decimals = 1)
    start_times = start_times[1:-1] # let's remove first and last elements to be safe
    for start_index, start in enumerate(start_times):
        selected_duration = resulting_durations[randrange(len(resulting_durations))]
        selected_sample_rate = sampling_rates[randrange(len(sampling_rates))]
        output_file = f"../data/sliced/{file_index}_{int(start_index*math.ceil(file_time + delta))}_{selected_duration}_{selected_sample_rate}.wav"
        ffmpeg_commands += [f"\
        ffmpeg -y -i {file} \
            -ss {start} \
            -t {selected_duration} \
            -ab 320k \
            -ar {selected_sample_rate} \
            -ac 1 \
            -vn {output_file} \
            && chown 1000:1000 {output_file} \
            "]

In [4]:
" ".join(ffmpeg_commands[600].split())

'ffmpeg -y -i ../data/source/0.mkv -ss 3606 -t 5.05 -ab 320k -ar 25718 -ac 1 -vn ../data/sliced/0_3600_5.05_25718.wav && chown 1000:1000 ../data/sliced/0_3600_5.05_25718.wav'

In [5]:
%%time
### SET THREADS NUMBER 
nproc = 6
bash_command = "sleep 5" # for tests

semaphore = threading.Semaphore(nproc)
def run_command(cmd):
    with semaphore:
        os.system(cmd)
threads = []

for task in ffmpeg_commands[]:
    threads += [threading.Thread(target=run_command, args=(task,))]

for j in threads:
    j.start()
for j in threads:
    j.join()

CPU times: user 6.73 ms, sys: 30.2 ms, total: 36.9 ms
Wall time: 2min 7s
